In [1]:
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

from mlxtend.classifier import StackingCVClassifier

from sklearn.datasets import load_iris
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler

import numpy as np

In [2]:
iris = load_iris()
X = iris.data
y = iris.target

In [3]:
RANDOM_SEED = 2

In [4]:
lr_pipe = Pipeline([('prep', StandardScaler()),
                    ('lr', LogisticRegression())])
knn_pipe = Pipeline([('prep', StandardScaler()),
                     ('knn', KNeighborsClassifier())])
rf = RandomForestClassifier(random_state=RANDOM_SEED)

In [8]:
np.random.seed(RANDOM_SEED)
sclf = StackingCVClassifier(classifiers=[lr_pipe, knn_pipe, rf], 
                            cv=3,
#                             use_features_in_secondary=True, 
                            use_probas=True,
                            meta_classifier=LogisticRegression(random_state=RANDOM_SEED))

In [6]:
sclf.get_params().keys()

dict_keys(['pipeline-1', 'pipeline-2', 'randomforestclassifier', 'pipeline-1__steps', 'pipeline-1__prep', 'pipeline-1__lr', 'pipeline-1__prep__copy', 'pipeline-1__prep__with_mean', 'pipeline-1__prep__with_std', 'pipeline-1__lr__C', 'pipeline-1__lr__class_weight', 'pipeline-1__lr__dual', 'pipeline-1__lr__fit_intercept', 'pipeline-1__lr__intercept_scaling', 'pipeline-1__lr__max_iter', 'pipeline-1__lr__multi_class', 'pipeline-1__lr__n_jobs', 'pipeline-1__lr__penalty', 'pipeline-1__lr__random_state', 'pipeline-1__lr__solver', 'pipeline-1__lr__tol', 'pipeline-1__lr__verbose', 'pipeline-1__lr__warm_start', 'pipeline-2__steps', 'pipeline-2__prep', 'pipeline-2__knn', 'pipeline-2__prep__copy', 'pipeline-2__prep__with_mean', 'pipeline-2__prep__with_std', 'pipeline-2__knn__algorithm', 'pipeline-2__knn__leaf_size', 'pipeline-2__knn__metric', 'pipeline-2__knn__metric_params', 'pipeline-2__knn__n_jobs', 'pipeline-2__knn__n_neighbors', 'pipeline-2__knn__p', 'pipeline-2__knn__weights', 'randomforestcl

In [17]:
params = {'pipeline-1__lr__C': Real(1e-6, 1e6, prior='log-uniform'),
          'pipeline-1__lr__penalty': Categorical(['l1', 'l2']),
          'pipeline-2__knn__n_neighbors': Integer(1, 20),
          'randomforestclassifier__max_features': Integer(1, 4),
          'meta-logisticregression__C': Real(1e-6, 1e6, prior='log-uniform'),
          'meta-logisticregression__penalty': Categorical(['l1', 'l2'])}

In [20]:
opt = BayesSearchCV(sclf, search_spaces=params, n_jobs=-1, n_iter=200,
                    random_state=RANDOM_SEED,  cv=5)

In [21]:
opt.fit(X, y)

In [22]:
opt.best_estimator_

StackingCVClassifier(classifiers=[Pipeline(steps=[('prep', StandardScaler(copy=True, with_mean=True, with_std=True)), ('lr', LogisticRegression(C=1000000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=No..._estimators=10, n_jobs=1, oob_score=False, random_state=2,
            verbose=0, warm_start=False)],
           cv=2,
           meta_classifier=LogisticRegression(C=1000000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=2,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
           shuffle=True, stratify=True, use_features_in_secondary=False,
           use_probas=False, verbose=0)

In [24]:
import pandas as pd
cv_results = pd.DataFrame(opt.cv_results_)

In [26]:
pd.options.display.max_columns = None
cv_results.sort_values('mean_test_score', ascending=False)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_meta-logisticregression__C,param_meta-logisticregression__penalty,param_pipeline-1__lr__C,param_pipeline-1__lr__penalty,param_pipeline-2__knn__n_neighbors,param_randomforestclassifier__max_features,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
13,0.095979,0.002354,0.966667,0.978333,1000000.000000,l1,1000000.000000,l1,20,1,"{'meta-logisticregression__C': 1000000.0, 'met...",1,0.966667,0.958333,1.000000,0.975000,0.933333,0.975000,0.933333,1.000000,1.000000,0.983333,0.021723,0.000162,0.029814,0.013540
9,0.078904,0.002362,0.960000,0.978333,32372.886936,l1,119922.624903,l2,8,3,"{'meta-logisticregression__C': 32372.8869361, ...",2,0.966667,0.975000,0.966667,0.966667,0.933333,0.983333,0.933333,1.000000,1.000000,0.966667,0.004879,0.000166,0.024944,0.012472
22,0.070162,0.002204,0.960000,0.981667,1.276481,l1,3.422541,l1,12,1,"{'meta-logisticregression__C': 1.2764809536, '...",1,0.966667,0.975000,1.000000,0.983333,0.900000,1.000000,0.933333,0.975000,1.000000,0.975000,0.006964,0.000222,0.038873,0.009718
18,0.081357,0.002498,0.960000,0.983333,2.224600,l2,0.000001,l1,10,1,"{'meta-logisticregression__C': 2.22460005073, ...",1,0.966667,0.991667,1.000000,0.975000,0.933333,0.983333,0.900000,0.983333,1.000000,0.983333,0.006316,0.000125,0.038873,0.005270
16,0.083328,0.002512,0.960000,0.978333,29.758500,l1,38.832415,l1,13,3,"{'meta-logisticregression__C': 29.7585003886, ...",1,0.966667,0.983333,0.966667,0.958333,0.933333,0.975000,0.933333,0.983333,1.000000,0.991667,0.006453,0.000060,0.024944,0.011304
14,0.089408,0.002221,0.960000,0.976667,1000000.000000,l1,884.073301,l2,7,2,"{'meta-logisticregression__C': 1000000.0, 'met...",2,0.966667,0.975000,0.966667,0.966667,0.933333,0.975000,0.933333,0.991667,1.000000,0.975000,0.017813,0.000186,0.024944,0.008165
31,0.069483,0.002048,0.960000,0.985000,4.028194,l2,1000000.000000,l1,19,4,"{'meta-logisticregression__C': 4.02819444717, ...",1,0.966667,0.958333,0.966667,0.991667,0.933333,0.991667,0.933333,1.000000,1.000000,0.983333,0.007082,0.000213,0.024944,0.014337
48,0.074298,0.002321,0.960000,0.975000,2400.113641,l2,0.000001,l1,6,4,"{'meta-logisticregression__C': 2400.11364056, ...",1,0.966667,0.966667,0.966667,0.958333,0.933333,0.991667,0.933333,0.983333,1.000000,0.975000,0.002843,0.000271,0.024944,0.011785
11,0.090264,0.002454,0.960000,0.976667,62.077833,l1,69.616213,l1,13,3,"{'meta-logisticregression__C': 62.0778331506, ...",2,0.966667,0.958333,0.966667,0.966667,0.933333,0.991667,0.933333,0.991667,1.000000,0.975000,0.013626,0.000048,0.024944,0.013333
28,0.075514,0.002514,0.960000,0.986667,0.206061,l1,0.000001,l1,20,4,"{'meta-logisticregression__C': 0.206061444345,...",1,0.966667,0.991667,0.966667,0.991667,0.933333,0.991667,0.933333,0.983333,1.000000,0.975000,0.001547,0.000142,0.024944,0.006667
